# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [4]:
# Initialize and constants

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'llama3.2'
openai = OpenAI()

In [7]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:14<00:00, 26.81s/it]


In [8]:
len(deals)

50

In [9]:
deals[44].describe()

"Title: Temu Lightning Deals: Up to 89% off + shipping varies\nDetails: Shop hundreds of deals in many different categories. We've pictured the Solar LED Deck Lights 32-Pack for $12 ($80 off). Shipping varies based on warehouse location and product. Shop Now at Temu\nFeatures: \nURL: https://www.dealnews.com/Temu-Lightning-Deals-Up-to-89-off-shipping-varies/21788534.html?iref=rss-c196"

In [10]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [11]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [12]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Seenda Bluetooth Waterproof Karaoke Machine w/ 2 Wireless Mics for $70 + free shipping
Details: It's $129 under list price. Buy Now at Walmart
Features: dual 6.5" woofers 1.5" tweeter 50W power output supports Bluetooth 5.3, USB flash drives, TF cards, 3.5mm AUX, wireless/wired microphones, guitar input, TWS function, and FM radio connectivity built-in RGB light sys

In [16]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="llama3.2",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [17]:
result = get_recommendations()

In [18]:
len(result.deals)

4

In [19]:
result.deals[1]

Deal(product_description='Euhomy Smart Countertop Nugget Ice Maker for $195 + free shipping', price=195.0, url='https://www.dealnews.com/Euhomy-Smart-Countertop-Nugget-Ice-Maker-for-195-free-shipping/21788579.html?iref=rss-c196')

In [2]:
from agents.scanner_agent import ScannerAgent

In [5]:
agent = ScannerAgent()
result = agent.scan()

In [6]:
result

DealSelection(deals=[Deal(product_description='JMGO N1S 4K Smart Portable Projector', price=799.0, url='.../jmgo-n1s-4k-smart-portable-projector-for-799-free-accessory-free-shipping'), Deal(product_description='Euhomy Smart Countertop Nugget Ice Maker', price=195.0, url='.../euhomy-smart-countertop-nugget-ice-maker-for-195-free-shipping'), Deal(product_description='Hyper Tough Hyper Stack Tool Storage System', price=59.0, url='.../hyper-tough-hyper-stack-tool-storage-system-for-59-free-shipping'), Deal(product_description='GVDV Heavy Duty Collapsible Folding Wagon', price=57.0, url='.../gvdv-heavy-duty-collapsible-folding-wagon-for-57-free-shipping'), Deal(product_description='Wyze Noise-Cancelling Headphones', price=40.0, url='.../wyze-noise-cancelling-headphones-for-40-free-shipping')])

In [7]:
print(result)

deals=[Deal(product_description='JMGO N1S 4K Smart Portable Projector', price=799.0, url='.../jmgo-n1s-4k-smart-portable-projector-for-799-free-accessory-free-shipping'), Deal(product_description='Euhomy Smart Countertop Nugget Ice Maker', price=195.0, url='.../euhomy-smart-countertop-nugget-ice-maker-for-195-free-shipping'), Deal(product_description='Hyper Tough Hyper Stack Tool Storage System', price=59.0, url='.../hyper-tough-hyper-stack-tool-storage-system-for-59-free-shipping'), Deal(product_description='GVDV Heavy Duty Collapsible Folding Wagon', price=57.0, url='.../gvdv-heavy-duty-collapsible-folding-wagon-for-57-free-shipping'), Deal(product_description='Wyze Noise-Cancelling Headphones', price=40.0, url='.../wyze-noise-cancelling-headphones-for-40-free-shipping')]
